In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from collections import Counter
from sklearn.cluster import KMeans

# Загрузка изображения
image_path = 'Rotated4.jpg'
image = Image.open(image_path)

# Вычисление новых размеров с сохранением пропорций
width = 1100
ratio = width / float(image.size[0])
height = int(float(image.size[1]) * ratio)

# Изменение размера изображения с использованием LANCZOS
resized_image = image.resize((width, height), Image.Resampling.LANCZOS)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_resized{ext}"
new_image_path = os.path.join('resized_data/', new_filename)

# Сохранение измененного изображения
resized_image.save(new_image_path)

# Опционально: показ изображения
resized_image.show()

name1 = new_image_path

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            if abs(angle1 - angle2) > angle_threshold:
                continue

            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            group.append(line2)
            used_indices.add(j)

        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Функция для нахождения точки пересечения двух линий
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    A1 = y2 - y1
    B1 = x1 - x2
    C1 = A1 * x1 + B1 * y1

    A2 = y4 - y3
    B2 = x3 - x4
    C2 = A2 * x3 + B2 * y3

    determinant = A1 * B2 - A2 * B1

    if determinant == 0:
        return None  # Линии параллельны
    else:
        x = (B2 * C1 - B1 * C2) / determinant
        y = (A1 * C2 - A2 * C1) / determinant
        return int(x), int(y)

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Функция для получения доминирующего цвета
def get_dominant_color(image_path, num_colors=1):
    image = Image.open(image_path)
    image = image.resize((150, 150))
    image_array = np.array(image)
    pixels = image_array.reshape(-1, image_array.shape[-1])
    if pixels.shape[1] == 4:
        pixels = pixels[:, :3]
    color_counts = Counter(map(tuple, pixels))
    dominant_color = color_counts.most_common(num_colors)[0][0]
    return dominant_color

# Функция для проверки параллельности линий
def are_lines_parallel(lines, angle_threshold=2):
    if len(lines) < 2:
        return False

    angles = []
    for line in lines[:2]:  # Проверяем только первые две линии
        x1, y1, x2, y2 = line
        angle = calculate_angle(x1, y1, x2, y2)
        angles.append(angle)

    return abs(angles[0] - angles[1]) <= angle_threshold

# Основной код обработки изображения
def process_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Применение Canny edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Применение Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

    # Преобразование линий в нужный формат
    if lines is None:
        print("Линии не обнаружены. Возвращаем исходное изображение.")
        return image

    lines = [line[0] for line in lines]

    # Объединение близких линий
    merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

    # Проверяем, параллельны ли линии
    if are_lines_parallel(merged_lines):
        print("Линии параллельны или отклоняются не более чем на 2 градуса. Применяем поворот.")
        return apply_rotation(image, image_path, merged_lines)
    else:
        print("Линии не параллельны. Применяем коррекцию перспективы.")
        return apply_perspective_correction(image, image_path, merged_lines)

# Функция для применения поворота
def apply_rotation(image, image_path, lines):
    # Вычисляем средний угол поворота
    if len(lines) >= 2:
        line1 = lines[0]
        line2 = lines[1]

        angle1 = calculate_angle(*line1)
        angle2 = calculate_angle(*line2)
        average_angle = (angle1 + angle2) / 2
    else:
        average_angle = 0

    # Получаем доминирующий цвет
    dominant_color = get_dominant_color(image_path)
    if isinstance(dominant_color, (tuple, list, np.ndarray)):
        dominant_color = tuple(map(int, dominant_color))
    else:
        dominant_color = (int(dominant_color),) * 3

    # Вычисляем матрицу поворота
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, average_angle, 1.0)

    # Вычисляем новые размеры холста после поворота
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Корректируем матрицу поворота для учета нового холста
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    # Применяем поворот к изображению с новыми размерами холста
    rotated_image = cv2.warpAffine(
        image,
        rotation_matrix,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=dominant_color
    )

    return rotated_image

# Функция для применения коррекции перспективы
def apply_perspective_correction(image, image_path, lines):
    # Создаем копию изображения для рисования линий
    output_image = image.copy()

    # Получаем высоту и ширину изображения
    image_height, image_width, _ = image.shape

    # Словарь для хранения центров линий
    line_centers = {}

    # Отрисовка всех линий и сохранение их центров
    for i, line in enumerate(lines):
        x1, y1, x2, y2 = line
        center = calculate_center(x1, y1, x2, y2)
        line_centers[i] = center

    # Фильтрация горизонтальных линий (угол от -40 до 40 градусов)
    middle_third_width_start = image_width // 3
    middle_third_width_end = 2 * image_width // 3
    min_horizontal_line_length = image_width / 4

    # Фильтр исключения линий с центрами ближе 20 пикселей к границам
    horizontal_lines = {
        line_id: center for line_id, center in line_centers.items()
        if (middle_third_width_start <= center[0] <= middle_third_width_end) and
           (calculate_line_length(*lines[line_id]) >= min_horizontal_line_length) and
           (-40 <= calculate_angle(*lines[line_id]) <= 40) and
           (20 <= center[0] <= image_width - 20) and  # Фильтр по X
           (25 <= center[1] <= image_height - 20)  # Фильтр по Y
    }

    # Фильтрация вертикальных линий (угол от 50 до 130 градусов)
    middle_third_height_start = image_height // 3
    middle_third_height_end = 2 * image_height // 3
    min_vertical_line_length = image_height / 9

    # Фильтр исключения линий с центрами ближе 20 пикселей к границам
    vertical_lines = {
        line_id: center for line_id, center in line_centers.items()
        if (middle_third_height_start <= center[1] <= middle_third_height_end) and
           (calculate_line_length(*lines[line_id]) >= min_vertical_line_length) and
           (50 <= abs(calculate_angle(*lines[line_id])) <= 130) and
           (20 <= center[0] <= image_width - 20) and  # Фильтр по X
           (20 <= center[1] <= image_height - 20)  # Фильтр по Y
    }

    # Нахождение двух горизонтальных и двух вертикальных линий
    if horizontal_lines and vertical_lines:
        # Выбираем две горизонтальные линии (с минимальным и максимальным Y)
        min_y_line = min(horizontal_lines.items(), key=lambda item: item[1][1])
        max_y_line = max(horizontal_lines.items(), key=lambda item: item[1][1])

        # Выбираем две вертикальные линии (с минимальным и максимальным X)
        min_x_line = min(vertical_lines.items(), key=lambda item: item[1][0])
        max_x_line = max(vertical_lines.items(), key=lambda item: item[1][0])

        # Получаем координаты линий
        h_line1 = lines[min_y_line[0]]
        h_line2 = lines[max_y_line[0]]
        v_line1 = lines[min_x_line[0]]
        v_line2 = lines[max_x_line[0]]

        # Находим точки пересечения
        intersection_A = line_intersection(h_line1, v_line1)  # Левая верхняя
        intersection_B = line_intersection(h_line2, v_line1)  # Левая нижняя
        intersection_C = line_intersection(h_line1, v_line2)  # Правая верхняя
        intersection_D = line_intersection(h_line2, v_line2)  # Правая нижняя

        # Проверка, что точки пересечения найдены
        if intersection_A and intersection_B and intersection_C and intersection_D:
            # Увеличение изображения
            padding = 0  # Отступ для увеличения изображения
            output_image = cv2.copyMakeBorder(output_image, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=(0, 0, 0))

            # Смещение координат точек пересечения
            intersection_A = (intersection_A[0] + padding, intersection_A[1] + padding)
            intersection_B = (intersection_B[0] + padding, intersection_B[1] + padding)
            intersection_C = (intersection_C[0] + padding, intersection_C[1] + padding)
            intersection_D = (intersection_D[0] + padding, intersection_D[1] + padding)

            # Вычисление центра изображения
            center_x = image_width // 2
            center_y = image_height // 2

            # Вычисление реальных размеров прямоугольника (примерные значения)
            real_width = 1.0  # Предполагаемая реальная ширина (в условных единицах)
            real_height = 1.0  # Предполагаемая реальная высота (в условных единицах)

            # Масштабирование реальных размеров для отображения на изображении
            scale_factor = 100  # 1 метр = 100 пикселей
            rect_width_px = int(real_width * scale_factor)
            rect_height_px = int(real_height * scale_factor)

            # Вычисление координат реального прямоугольника
            rect_top_left = (center_x - rect_width_px // 2, center_y - rect_height_px // 2)
            rect_bottom_right = (center_x + rect_width_px // 2, center_y + rect_height_px // 2)

            # Координаты точек A'', B'', C'', D'' для масштабированного прямоугольника
            scaled_rect_A_double_prime = (rect_top_left[0], rect_top_left[1])
            scaled_rect_B_double_prime = (rect_top_left[0], rect_bottom_right[1])
            scaled_rect_C_double_prime = (rect_bottom_right[0], rect_top_left[1])
            scaled_rect_D_double_prime = (rect_bottom_right[0], rect_bottom_right[1])

            # Вычисление преобразования перспективы
            # Исходные точки (A, B, C, D)
            pts1 = np.float32([intersection_A, intersection_B, intersection_C, intersection_D])

            # Целевые точки (A'', B'', C'', D'')
            pts2 = np.float32([scaled_rect_A_double_prime, scaled_rect_B_double_prime,
                              scaled_rect_C_double_prime, scaled_rect_D_double_prime])

            # Вычисляем матрицу преобразования перспективы
            M = cv2.getPerspectiveTransform(pts1, pts2)

            # Определяем размеры выходного изображения
            h, w = image.shape[:2]
            corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
            transformed_corners = cv2.perspectiveTransform(np.array([corners]), M)[0]

            # Вычисляем новые размеры изображения
            x_min = int(np.min(transformed_corners[:, 0]))
            x_max = int(np.max(transformed_corners[:, 0]))
            y_min = int(np.min(transformed_corners[:, 1]))
            y_max = int(np.max(transformed_corners[:, 1]))

            new_width = x_max - x_min
            new_height = y_max - y_min

            # Создаем матрицу сдвига для корректировки обрезания
            shift_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])

            # Объединяем матрицу преобразования и сдвига
            M_shifted = shift_matrix @ M

            # Определяем доминирующий цвет
            def get_dominant_color_kmeans(image, k=1):
                pixels = image.reshape(-1, 3)
                kmeans = KMeans(n_clusters=k, n_init=10)
                kmeans.fit(pixels)
                dominant_color = kmeans.cluster_centers_[0].astype(int)
                return dominant_color

            dominant_color = get_dominant_color_kmeans(image)

            # Применяем преобразование перспективы
            warped_image = cv2.warpPerspective(image, M_shifted, (new_width, new_height))

            # Обработка добавленных областей
            mask = (warped_image == 0).all(axis=2).astype(np.uint8) * 255
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            min_area = 40
            for contour in contours:
                if cv2.contourArea(contour) >= min_area:
                    contour_mask = np.zeros_like(mask)
                    cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)
                    kernel = np.ones((5, 5), np.uint8)
                    expanded_contour_mask = cv2.dilate(contour_mask, kernel, iterations=1)
                    warped_image[expanded_contour_mask == 255] = dominant_color
                    border_mask = cv2.Canny(contour_mask, 100, 200)
                    border_mask = cv2.dilate(border_mask, None, iterations=2)
                    blurred = cv2.GaussianBlur(warped_image, (15, 15), 0)
                    warped_image[border_mask == 255] = blurred[border_mask == 255]

            return warped_image

    # Если не удалось найти пересечения, возвращаем исходное изображение
    return image

# Обработка изображения
result_image = process_image(name1)

# Сохранение результата
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
output_filename = f"{name}_processed{ext}"
output_path = os.path.join('Output_data/', output_filename)
cv2.imwrite(output_path, result_image)

# Показ результата
cv2.imshow('Processed Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"Обработка завершена. Результат сохранен в файл '{output_path}'.")